In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_data_set,mnist_info=tfds.load(name='mnist',as_supervised=True,with_info=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
mnist_train,mnist_test=mnist_data_set['train'],mnist_data_set['test']

num_validation_sample=0.1 * mnist_info.splits['train'].num_examples
num_validation_sample=tf.cast(num_validation_sample,tf.int64)

num_test_sample=0.1 * mnist_info.splits['test'].num_examples
num_test_sample=tf.cast(num_test_sample,tf.int64)

In [4]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [5]:
def scale(image,label):
  image=tf.cast(image,tf.float32)
  image/=255
  return image,label

In [6]:
scaled_train_validation_data=mnist_train.map(scale)

In [8]:
BUFFER_SIZE=10000

shuffled_train_and_validation_data=scaled_train_validation_data.shuffle(BUFFER_SIZE)
validation_data=shuffled_train_and_validation_data.take(num_validation_sample)

train_data=shuffled_train_and_validation_data.skip(num_validation_sample)

BATCH_SIZE=100
train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_sample)

validation_input,validation_target=next(iter(validation_data))

In [17]:
input_size=784
output_size=10
hidden_layer_size=50

model=tf.keras.Sequential([
                           tf.keras.layers.Flatten(input_shape = (28,28,1)),
                           tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                           tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                           tf.keras.layers.Dense(output_size, activation='softmax')
])
model.compile(optimizer='adam',loss='sparse_categorical_entry',metrics=['accuracy'])

In [21]:
model.fit(train_data, epochs=5, validation_data=(validation_input, validation_target), verbose =2)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)